In [ ]:
import numpy as np
import torch
import torch.nn as nn

"""
순서
어텐션
MLP(피드포워드)
인코더 블록
인코더
"""
"""
나머지 주석은 나중에
"""

class MultiheadSelfAttention(nn.Module):

  def __init__(self, d_model: int, num_heads: int, dropout_rate: float=0.1):
    super().__init__()
    self.num_heads = num_heads # 헤드의 수를 num_heads에 저장
    self.d_model = d_model # D차원을 d_model에 저장
    self.head_dim = d_model // self.num_heads # d차원을 헤드의 수로 나누어서 각 헤드의 차원을 구함. 이를 위해 D차원은 num_heads로 나누어 떨어져야 함

    self.query = nn.Linear(d_model, d_model) # 아래의 선형연산을 위한 선형레이어들
    self.key = nn.Linear(d_model, d_model)
    self.value = nn.Linear(d_model, d_model)

    self.softmax = nn.Softmax(dim=-1)

    self.dropout = nn.Dropout(p=dropout_rate)

    self.WO = nn.Linear(d_model, d_model) # W^O차원

  def forward(self, inputs_q, inputs_k=None, inputs_v=None, mask=None):

    """
    입력 받는 텐선 inputs_q는 [batch_sizes..., length, features]의 형태로 배치 사이즈, 길이(N+1), D차원(768)으로 이루어져있음.
    """

    if inputs_k is None:
      inputs_k = inputs_q # K나 V에 해당하는 텐서를 받지 못하였을 때, Q텐서로 K, V텐서를 만들겠다는 의미로 같은 토큰에서 Q, K, V를 뽑아내는 것
    if inputs_v is None:
      inputs_v = inputs_k

    batch, len, d_size = inputs_q.shape

    Q = self.query(inputs_q)  # W^q의 역할로 입력된 토큰에 가중치를 부여하는 선형연산
    K = self.key(inputs_k)    # W^k의 역할로 입력된 토큰에 가중치를 부여하는 선형연산
    V = self.value(inputs_v)  # W^v의 역할로 입력된 토큰에 가중치를 부여하는 선형연산

    Q = Q.view(batch, len, self.num_heads, self.head_dim)  # 텐서 Q에 대해서 마지막 차원에 대해 변환을 하여 [배치 사이즈, 길이, 헤드 수, 헤드 차원]으로 변환함
    K = K.view(batch, len, self.num_heads, self.head_dim)  # 이유는 멀티 헤드 셀프 어텐션을 위해 헤드의 차원을 나누어서 헤드별로 어텐션이 될 수 있게 만듬
    V = V.view(batch, len, self.num_heads, self.head_dim)

    atten = torch.einsum("bqhd,bkhd->bhqk", Q, K)
    # Q와 K차원이 [배치사이즈, 길이, 헤드수, 헤드차원]으로 되어 있는데 이를 Q와 K^T의 내적을 배치와 헤드는 고정시켜두고
    # q,d 와 k, d에 대해 d에 대해 내적하여 [배치사이즈, 헤드수, q길이, k길이] 형태로 만드는 것
    # 이는 Q에 대한 K의 유사도가 됨
    atten /= torch.sqrt(torch.tensor(self.head_dim))
    # d_k 차원을 어텐션 스코어를 루트 d_k(헤드 차원)로 나누어 스케일 내적을 마무리

    if mask is not None:
      atten = torch.where(mask == 0, torch.tensor(-1e9).to(atten.device), atten)
    # mask가 1 or 0인거에 따라서 0이면 -1e9(-10^9)을 넣어주고 아니면 원래 값을 넣어줌
    # 이후 소프트맥스할 때 마스킹된 부분이 0언저리가 됨.

    atten_weights = self.softmax(atten)
    # 이후 어텐션 스코어에 소프트맥스를 해서 어텐션 분포를 얻음
    atten_weights = self.dropout(atten_weights)

    atten_output = torch.einsum("bhqk,bkhd->bqhd", atten_weights, V)
    # 이번에는 위에서 구한 [배치사이즈, 헤드수, q길이, k길이]형태의 어텐션 분포와 [배치 사이즈, k길이, 헤드 수, 헤드 차원]형태의 value 텐서를 곱하는데
    # k길이에 대해 내적하여 [배치 사이즈, q길이, 헤드수, 헤드 차원] 형태로 만듬
    # 이는 Q에 대한 K의 유사도에 맞추어 K에 대한 실제 값인 헤드 차원 부분이 내적되면서 벨류의 텐서에 어텐션 분포가 반영이 된다

    atten_output = atten_output.reshape(batch, len, self.num_heads * self.head_dim)
    # 다시 [배치사이즈, 길이, D차원] 형태로 변환함. 다시 합치는 Concat부분
    output = self.WO(atten_output)
    # 변환한 후 가중치 행렬 W^O를 취하기 위해서 학습 가능한 가중치를 레이어로 부여함

    return output

class FFNBlock(nn.Module):

  """
  피드 포워드 네트워크를 위한 블록.
  리니어 gelu 드롭아웃 리니어 드롭아웃의 형태로 원본의 순서를 따라함
  ViT 논문에사 relu대신 gelu를 썼가에 재현하고자 함
  """

  def __init__(self, d_model: int, mlp_dim: int, dropout_rate: float=0.1):
    super().__init__()
    self.d_model = d_model
    self.mlp_dim = mlp_dim
    self.linear1 = nn.Linear(d_model, mlp_dim)
    self.linear2 = nn.Linear(mlp_dim, d_model)
    self.gelu = nn.GELU()
    self.dropout = nn.Dropout(p=dropout_rate)

  def forward(self, x):
    x = self.linear1(x)
    x = self.gelu(x)
    x = self.dropout(x)
    x = self.linear2(x)
    y = self.dropout(x)

    return y

class EncoderBlock(nn.Module):
  """
    해야할 거: LN갈기고, MSA갈기고 skip-connection갈기기
    논문의 순서대로 인풋을 레이어정규화 이후에 MSA하고 이후 나온 값을 다시 인풋과 더하는 스킵 커넥션을 함
    이후 똑같이 정규화, 피드포워드, 스킵커넥션의 순서이다.
  """
  def __init__(self, d_model: int, num_heads: int, mlp_dim: int, dropout_rate: float=0.1):
    super().__init__()
    self.num_heads = num_heads
    self.d_model = d_model
    self.mlp_dim = mlp_dim
    self.LN = nn.LayerNorm(d_model)
    self.LN2 = nn.LayerNorm(d_model)
    self.MSA = MultiheadSelfAttention(d_model=d_model, num_heads=num_heads, dropout_rate=dropout_rate)
    self.FFN = FFNBlock(d_model=d_model, mlp_dim=mlp_dim, dropout_rate=dropout_rate)
    self.dropout = nn.Dropout(p=dropout_rate)



  def forward(self, inputs):
    z = self.LN(inputs) # LN(레이어 정규화)적용
    z = self.MSA(z) # MSA(멀티헤드셀프어텐션)적용
    z = self.dropout(z)
    z = z + inputs # 스킵커넥션 적용

    ffn = self.LN2(z) # LN적용
    ffn = self.FFN(ffn) # FFN적용
    # 이후 스킵커넥션을 적용한 값을 반환
    return z+ffn

class Encoder(nn.Module):

  """
  해야할 거: 포지셔널 임베딩 한 거를 L개의 레이어에 넣기
  L개의 레이어에 넣기 위해서는 L개의 모듀을 먼저 선언해야함
  원본 논문의 코드는 flax.linen을 썼는데 가장 큰 차이점은 torch는 객체로 되어있어 객체가 __init__에서 선언되어
  있어야 이후 forward가 작동 가능하고 linen은 함수로 되어 있어서 @nn.compact을 쓰고 __call__ 부분에 쓰고싶은 함수를 쓰면 되는 형식임
  이걸 말한 이유는 원본에서는 그냥 for문을 썼는데 똑같이 하면 같은 블록(레이어)이 반복되며 나오는거라 L개의 디코더블록이 아니라
  디코더 블록이 L번 학습을 진행하는 것임. 이걸 해결하기위해 nn.ModuleList를 써서 동적으로 묘듈을 할당해서 L개의 디코더블록을 만들 수 있음
  """
  def __init__(self, d_model: int, num_layers: int, num_heads: int, mlp_dim: int, dropout_rate: float=0.1):
    super().__init__()
    self.num_layers = num_layers
    self.num_heads = num_heads
    self.d_model = d_model
    self.mlp_dim = mlp_dim
    self.LN = nn.LayerNorm(d_model)
    self.layers = nn.ModuleList()
    for i in range(num_layers):
      self.layers.append(EncoderBlock(d_model=d_model, num_heads=num_heads, mlp_dim=mlp_dim, dropout_rate=dropout_rate))

  def forward(self, x):
    for layer in self.layers:
      x = layer(x)
    CLS = x[:, 0]
    CLS = self.LN(CLS)
    return CLS